In [1]:
import numpy as np

In [2]:
import pandas as pd

from collections import Counter

import itertools

In [3]:
import sympy as sym

In [14]:
import glob

import tqdmb

In [88]:
import repeated_play

In [22]:
def seperate_equilibria_and_non(df):
    per_player = df.groupby('ID')['condition B'].all()

    eq_players_ID = per_player[per_player == True].index

    noneq_players_ID = per_player[per_player == False].index
    
    return df[df['ID'].isin(eq_players_ID)], df[df['ID'].isin(noneq_players_ID)]

In [69]:
deterministic_strategies = list(
    itertools.product([0, 1], repeat=2 ** (2 * 2))
    )

### Numerical Nash for $n=2$ and $R = .6$, and $R=.3$

In [116]:
files = glob.glob("../data/prisoners_dilemma_n_three/*csv")

In [117]:
columns = (
    ["", "ID"]
    + [f"p{i+1}" for i in range(16)]
    + ["label", "Sp", "Sq", "condition A", "condition B", "R", "P"]
)

In [118]:
columns3 = ["ID"] + [f"p{i+1}" for i in range(8)] +  ["label", "Sp", "Sq", "condition A", "condition B", "R", "P"]

In [119]:
dfs = []

for file in tqdm.tqdm(files):

    dfs.append(pd.read_csv(file, index_col=0, names=columns3))

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:31<00:00, 314.64it/s]


In [120]:
df = pd.concat(dfs)

In [121]:
df.head()

,ID,p1,p2,p3,p4,p5,p6,p7,p8,label,Sp,Sq,condition A,condition B,R,P
0,3520,1,0.3843,0.12232,0.04912,0.96004,0.39895,0.46845,0.14888,N0,0.6,0.233992,True,True,0.6,0.1
1,3520,1,0.3843,0.12232,0.04912,0.96004,0.39895,0.46845,0.14888,N1,0.6,0.333161,True,True,0.6,0.1
2,3520,1,0.3843,0.12232,0.04912,0.96004,0.39895,0.46845,0.14888,N2,0.6,0.233992,True,True,0.6,0.1
3,3520,1,0.3843,0.12232,0.04912,0.96004,0.39895,0.46845,0.14888,N3,0.6,0.365147,True,True,0.6,0.1
4,3520,1,0.3843,0.12232,0.04912,0.96004,0.39895,0.46845,0.14888,N4,0.6,0.233992,True,True,0.6,0.1


In [122]:
eq, noneq = seperate_equilibria_and_non(df)

In [123]:
eq['ID'].nunique(), noneq['ID'].nunique()

(3742, 6258)

#### The pure set of strategies we need to check 

In [124]:
groups = noneq[noneq['condition A'] == False].groupby('ID')

In [125]:
sets, ids = [], []

for i, group in groups:
    sets.append(group['label'].values)
    ids.append(group['ID'].unique())

In [175]:
rest_sets = [s for s in sets if ('N0' not in s) and ('N3' not in s) and ('N57' not in s)
             and ('N69' not in s) and ('N17' not in s) and ('N111' not in s) and ('N59' not in s)]

In [176]:
flat_list = [item for sublist in rest_sets for item in sublist]

counter = Counter(flat_list)

In [177]:
len(rest_sets)

2

In [178]:
max(counter.values())

1

In [179]:
counter

Counter({'N1': 1,
         'N9': 1,
         'N16': 1,
         'N24': 1,
         'N129': 1,
         'N137': 1,
         'N144': 1,
         'N152': 1,
         'N43': 1,
         'N58': 1})

In [145]:
[s for s in sets if ('N0' not in s) and ('N17' not in s) and ('N84' not in s)  and ('N59' not in s) and ]

[array(['N57'], dtype=object),
 array(['N100', 'N101', 'N102', 'N103', 'N108', 'N109', 'N111', 'N116',
        'N117', 'N118', 'N119', 'N124', 'N125', 'N126', 'N127'],
       dtype=object),
 array(['N39', 'N47', 'N50', 'N54', 'N55', 'N58', 'N62', 'N63', 'N99',
        'N101', 'N102', 'N103', 'N107', 'N108', 'N109', 'N110', 'N111',
        'N114', 'N115', 'N116', 'N117', 'N118', 'N119', 'N122', 'N123',
        'N124', 'N125', 'N126', 'N127'], dtype=object),
 array(['N101', 'N102', 'N103', 'N107', 'N108', 'N109', 'N110', 'N111',
        'N114', 'N115', 'N116', 'N117', 'N118', 'N119', 'N121', 'N122',
        'N123', 'N124', 'N125', 'N126', 'N127'], dtype=object),
 array(['N110', 'N111', 'N126', 'N127'], dtype=object),
 array(['N102', 'N103', 'N111', 'N115', 'N118', 'N119', 'N122', 'N123',
        'N126', 'N127'], dtype=object),
 array(['N55', 'N63', 'N101', 'N102', 'N103', 'N108', 'N109', 'N110',
        'N111', 'N115', 'N116', 'N117', 'N118', 'N119', 'N122', 'N123',
        'N124', 'N125

In [180]:
R, P = sym.symbols("R, P")

p1, p2, p3, p4 = sym.symbols("p_1, p_2, p_3,p_4")

In [181]:
strategy = [ p1, p2, p1, p2, p3, p4, p3, p4,
             p1, p2, p1, p2, p3, p4, p3, p4]

In [182]:
pure_self_reactive = list(itertools.product([0, 1], repeat=4))

In [183]:
q1, q2, q3, q4 = pure_self_reactive[3]

coplayer =  [q1, q1, q2, q2, q1, q1, q2, q2,
             q3, q3, q4, q4, q3, q3, q4, q4]

In [184]:
M = repeated_play.transition_matrix_repeated_game(coplayer, strategy, memory="two", analytical=True)

In [185]:
ss = repeated_play.stationary_distribution(M, analytical=True)

In [189]:
(sum(ss @ sym.Matrix([R, 0, 1, P] * 4)).factor().subs({p1: 1}) - R)

-P*p_2/4 + P/4 + R*p_3/4 + R*p_4/4 - R + p_2/4 + 1/4